# Imports

In [1]:
from despliegue.loaders import *
from despliegue.nodos import *
from despliegue.contenedores import *
from despliegue.solvers import *
import pandas as pd
from os import path

# Constantes

Se definen las rutas de los datos, y también el `DataFrame` de la Red Móvil, ya que no posee la vacancia, y hay que asignarla a mano.

In [2]:
PATH_DATA = path.join("./data/")  # Path donde estarán los Excels
PATH_CLIENTES = path.join(PATH_DATA, "Direcciones_Colina.xlsx")  # Path de los clientes
PATH_FO = path.join(PATH_DATA, "CTO_Colina.xlsx")  # Path donde estará las CTO's
PATH_RM = path.join(PATH_DATA, "SITIOS 4G TDD.xlsx")  # Path donde estarán las antenas de RM

In [3]:
df_clientes = pd.read_excel(PATH_CLIENTES)
df_clientes = df_clientes[df_clientes.gl_lat_OK <= -1]  # limpiamos aquellos que sean de lat-lon=0
df_clientes.describe()

,pcm_area_tel,gl_lat_OK,gl_lon_OK
count,2.750000e+02,275.000000,275.000000
mean,2.286022e+08,-33.200679,-70.672530
std,1.034212e+06,0.013769,0.011696
min,2.240048e+08,-33.301086,-70.723406
25%,2.284422e+08,-33.207438,-70.679538
50%,2.284444e+08,-33.202714,-70.673384
75%,2.284477e+08,-33.192004,-70.666531
max,2.329671e+08,-33.173324,-70.632150


In [4]:
df_fo = pd.read_excel(PATH_FO)
df_fo.describe()

,id_pto_ftth,latitud_ok,longitud_ok,eqpt_capacity,eqpt_vg_qty,oc_ds,porcentaje_ocup_caja
count,3.484000e+03,3484.000000,3484.000000,3484.000000,3484.00000,3484.00000,3484.000000
mean,2.476223e+06,-33.202056,-70.673801,7.756889,5.74225,2.00287,23.659374
std,4.067262e+05,0.013274,0.009063,0.839911,1.56150,1.40341,17.531295
min,1.758698e+06,-33.222792,-70.711280,1.000000,1.00000,0.00000,0.000000
25%,2.120851e+06,-33.214408,-70.681072,8.000000,5.00000,1.00000,12.500000
50%,2.478504e+06,-33.203207,-70.673720,8.000000,6.00000,2.00000,25.000000
75%,2.828275e+06,-33.190197,-70.667940,8.000000,7.00000,3.00000,37.500000
max,3.177018e+06,-33.176260,-70.649690,9.000000,8.00000,5.00000,57.140000


In [5]:
df_rm = pd.read_excel(PATH_RM)
df_rm["vacancia"] = 30
df_rm.describe()

,LATITUD,LONGITUD,vacancia
count,99.000000,99.000000,99.0
mean,-33.504739,-70.714568,30.0
std,0.116812,0.102712,0.0
min,-33.810990,-71.225640,30.0
25%,-33.576260,-70.746720,30.0
50%,-33.509860,-70.695800,30.0
75%,-33.409805,-70.664650,30.0
max,-33.205890,-70.537380,30.0


In [6]:
df_rm.head()

,NEMONICO,LATITUD,LONGITUD,vacancia
0,CURAOD13,-33.402400,-71.128100,30
1,MONG1D13,-33.677625,-70.999925,30
2,CCHC1D13,-33.392800,-70.672800,30
3,CATC2D13,-33.407100,-70.681000,30
4,ALDEFD13,-33.533130,-70.635310,30


# Conexión a los Datos

Creamos las instancias de conexión a la base de datos. Observar que para `rm_db` le entregamos el `DataFrame` directamente.

In [7]:
cliente_db = ClienteDB(df=df_clientes, col_names=["pcm_area_tel", "gl_lat_OK", "gl_lon_OK"])
fo_db = OfertaDB(df=df_fo, col_names=["id_pto_ftth", "latitud_ok", "longitud_ok", "eqpt_vg_qty"])
rm_db = OfertaDB(df=df_rm)  # Las columnas justamente están ordenadas en [id, lat, lon, vac]

# Conjuntos de Nodos

Creamos los conjuntos de nodos utilizando los loaders de la parte anterior

In [8]:
nodos_oferta = NodosOferta(fo_db, rm_db)
nodos_demanda = NodosDemanda(cliente_db)

# Optimización

Creamos una instancia del tercer solver, que se corresponde con el siguiente modelo:
\begin{equation}
\left\{\begin{array}{rll}
\max & \sum_{i=1}^{n} \sum_{j\in N(i)} c_{ij} x_{ij} & \\
\mbox{s.a.} & \sum_{j\in N(i)} x_{ij} \leq O_i.vac & \forall i = 1:n \\
& \sum_{i \in N(j)} x_{ij} = 1 & \forall j = 1 : m \\
& x_{ij} \in \{0, 1\} &
\end{array}\right.
\end{equation}

Donde $N(i)$ son los vecinos del vértice $i$. Y consideramos a las aristas 
$$E = \{
ij \ | \ i \in FO, \ j=1:m, \ O_i.\mbox{dist}_1(D_j) \leq C_{FO}
\} \cup \{
ij \ | \ i \in RM, \ j=1:m, \ O_i.\mbox{dist}_2(D_j) \leq C_{RM}
\}$$

Además, interpretamos a la matriz $(c_{ij})_{ij}$ como una matriz de prioridad, donde se le dará mayor prioridad a la arista $ij$ en tanto tenga mayor $c_{ij}$. Consideramos la siguiente matriz:
\begin{equation}
c_{ij}
= \begin{cases}
\frac{a}{O_i.\mbox{dist}_1(D_j) + \varepsilon} & i \in FO \\
\frac{b}{O_i.\mbox{dist}_2(D_j) + \varepsilon} & i \in RM \\
0 & \mbox{en cualquier otro caso}
\end{cases}
\end{equation}

Intuitivamente, lo que hace es darle mayor prioridad a aquellos clientes que estén más cerca a cierta CTO o RM. 

In [9]:
modelo3 = Solver3(nodos_oferta, nodos_demanda, a=2, b=1, c_fo=150, c_rm=600, verbose=True)
modelo3.resolver()

Construyendo modelo...
Definiendo función objetivo...
Definiendo restricciones...
Empezando a resolver...
Estado: Optimal


In [10]:
modelo3.save(path.join(PATH_DATA, "resultado.xlsx"))

Salvando resultados...
Resultados Salvados!


Observamos que considerar este modelo reduce de casi un millón de variables a casi 4500, aumentando mucho la eficiencia en el tiempo.

In [11]:
modelo1 = Solver1(nodos_oferta, nodos_demanda)
modelo3 = Solver3(nodos_oferta, nodos_demanda)
len(modelo1.x), len(modelo3.x)

(985600, 4512)